In [67]:
import spatialmath as sm
import spatialgeometry as sg
import roboticstoolbox as rtb
from pydrake.solvers import MathematicalProgram, Solve
import swift
import time
import numpy as np


def jacobian_i_k_optimisation(robot, v, v_max=1.2):
    # jacobian inverse kinematics with optimisation
    J = robot.jacobe(robot.q)
    prog = MathematicalProgram()
    v_opt = prog.NewContinuousVariables(6, "v_opt")
    # Define the error term for the cost function
    error = J @ v_opt - v
    prog.AddCost(error.dot(error))
    # Add bounding box constraint for joint velocities
    lower_bounds = [-v_max] * 6  # Lower bounds for each joint velocity
    upper_bounds = [v_max] * 6   # Upper bounds for each joint velocity
    prog.AddBoundingBoxConstraint(lower_bounds, upper_bounds, v_opt)
    # Solve the optimization problem
    result = Solve(prog)
    return result.is_success(), result.GetSolution(v_opt)

def end_effector_base_position_from_tip(T_tip ,offset):
    """
    Calculate the end effector base position given the tip position and an arbitrary offset.
    
    :param tip_position: A tuple of (x, y, z) coordinates for the end effector tip.
    :param tip_orientation: A tuple of (roll, pitch, yaw) angles in degrees for the end effector tip orientation.
    :param offset: A tuple of (x_e, y_e, z_e, roll_e, pitch_e, yaw_e) representing the offset of the base from the tip.
    :return: The position of the end effector base as a tuple (x, y, z).
    """
    x_e, y_e, z_e, roll_e, pitch_e, yaw_e = offset
    T_offset = sm.SE3.RPY([roll_e, pitch_e, yaw_e], unit='deg') * sm.SE3(x_e, y_e, z_e)
    
    # Calculate the transformation matrix for the base
    T_base = T_tip * T_offset.inv()
    return T_base


In [68]:
class Lite6:
    def __init__(self, simulation, robot_ip = None, tcp_offset = None) -> None:
        self.virtual_robot = rtb.models.URDF.Lite6()
        self.simulation = simulation
        if self.simulation:
            self.simulation.add(self.virtual_robot, robot_alpha=True, collision_alpha=False)
        self.tcp_offset = tcp_offset
        self.real_robot = None
        if robot_ip:
            from xarm.wrapper import XArmAPI
            from robot import RobotMain
            self.real_robot = RobotMain(XArmAPI(robot_ip, baud_checkset=False))

    def move_to(self, dest, dt=0.05, gain=1, treshold=0.01):
        if self.tcp_offset:
            dest = end_effector_base_position_from_tip(dest, self.tcp_offset)
        if self.simulation:
            axes = sg.Axes(length=0.1, pose=dest)
            self.simulation.add(axes)
        if self.real_robot:
            self.real_robot.set_mode(4)
            self.real_robot.set_state(0)
        arrived = False
        while not arrived:
            v, arrived = rtb.p_servo(self.virtual_robot.fkine(self.virtual_robot.q), dest, gain=gain, threshold=treshold)
            qd = jacobian_i_k_optimisation(self.virtual_robot, v, v_max=1)[1]
            self.virtual_robot.qd = qd
            if self.real_robot:
                self._arm.vc_set_joint_velocity(qd, is_radian=True)
            if self.simulation:
                self.simulation.step(dt)
        if self.real_robot:
            self._arm.vc_set_joint_velocity([0, 0, 0, 0, 0, 0], is_radian=True)
        return arrived
    
    def get_pose(self):
        return self.virtual_robot.fkine(self.virtual_robot.q)
    
    def reset(self):
        self.virtual_robot.q = self.virtual_robot.qz
        self.simulation.step(0.1)

tcp_offset = [0, 0, 0.2, 0, 0, 0]
sim = swift.Swift()
sim.launch(realtime=True)
lite6 = Lite6(simulation=sim, tcp_offset=tcp_offset)


Opening in existing browser session.


In [69]:
lite6.reset()
x, y, z, roll, pitch, yaw = 0.2, 0.2, 0, -180, -180, 0
roll_rad, pitch_rad, yaw_rad = np.deg2rad([roll, pitch, yaw])
R = sm.SE3.RPY([roll_rad, pitch_rad, yaw_rad], order='xyz')
T = sm.SE3(x, y, z)
dest = T*R
lite6.move_to(dest)

True

Deserialization failed!


In [33]:
a = [1,2,3]
c, d, e = a
print(c, d, e)

1 2 3
